In [1]:
# Map
# https://telraam.net/

# API
# https://documenter.getpostman.com/view/8210376/TWDRqyaV#intro


In [1]:
# import packages
import json
import requests
import time

import pandas as pd
import altair as alt

In [2]:
# set url
url = "https://telraam-api.net/v1/reports/traffic"

In [3]:
#get apikey from txt file
filename_apikey = 'telraam_apikey.txt'

def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
        
api_key = get_file_contents(filename_apikey)        

In [5]:
# get data from api

# for the follwoing 3 segments
#https://telraam.net/en/location/9000004672/2023-05-02/2023-05-02   August-Bebel Allee
#https://telraam.net/en/location/9000004671/2023-05-02/2023-05-02   Bahnhofstr (Fahrradladen)
#https://telraam.net/en/location/9000004694/2023-05-02/2023-05-02   Grünauer Straße (Rathaus)


df_all=pd.DataFrame()

time_start="2023-05-02"
time_end="2023-05-03"

headers = {
  'X-Api-Key': api_key
}

# request data for each id and store it in a dataframe
for i in ['9000004672','9000004694','9000004671']:
    segment_id=i
    print (segment_id)
    
    payload = '{\r\n  \"level\": \"segments\",\r\n  \"format\": \"per-hour\",\r\n  \"id\": \"'+segment_id+'\",\r\n  \"time_start\": \"'+time_start+' 07:00:00Z\",\r\n  \"time_end\": \"'+time_end+' 00:00:00Z\"\r\n}'
    response = requests.request("POST", url, headers=headers, data=payload)
    
    #convert text/json to dataframe
    pretty_json = json.loads(response.text)
    df = pd.json_normalize(pretty_json['report'])
    
    df_all=pd.concat([df_all, df])  #append to single df
    
    time.sleep(2) #time to sleep to avoid timeout

9000004672
9000004694
9000004671


In [6]:
df_all.head()

,instance_id,segment_id,date,interval,uptime,heavy,car,bike,pedestrian,heavy_lft,...,car_rgt,bike_lft,bike_rgt,pedestrian_lft,pedestrian_rgt,direction,car_speed_hist_0to70plus,car_speed_hist_0to120plus,timezone,v85
0,-1,9000004672,2023-05-02T07:00:00.000Z,hourly,0.551944,11.921607,24.597149,58.192001,15.836871,1.480263,...,18.676608,23.357952,34.834049,5.555556,10.281316,1,"[17.7267016034, 64.7102871394, 17.5630112572, ...","[17.7267016034, 0.0, 52.6752732317, 12.0350139...",Europe/Berlin,18.5
1,-1,9000004672,2023-05-02T08:00:00.000Z,hourly,0.725000,24.093977,13.773038,40.332997,12.345699,7.132662,...,6.816573,16.689596,23.643401,9.605339,2.740361,1,"[20.8437188495, 39.7538643555, 39.402416795, 0...","[10.4218594247, 10.4218594247, 9.7968603588, 2...",Europe/Berlin,20.5
2,-1,9000004672,2023-05-02T09:00:00.000Z,hourly,0.669444,13.641115,20.899199,38.619067,13.294999,1.530612,...,11.864156,20.546928,18.072140,7.319127,5.975872,1,"[42.3059012515, 28.6229288455, 29.0711699031, ...","[27.8531557992, 14.4527454523, 0.0, 28.6229288...",Europe/Berlin,20.5
3,-1,9000004672,2023-05-02T10:00:00.000Z,hourly,0.655000,6.207964,21.565819,32.142094,22.918039,0.000000,...,13.672873,10.684511,21.457583,15.303737,7.614302,1,"[43.506658372, 41.974807028, 14.5185345999, 0....","[36.3483895737, 7.1582687984, 14.0589346771, 2...",Europe/Berlin,17.5
4,-1,9000004672,2023-05-02T11:00:00.000Z,hourly,0.660278,7.378531,32.622229,66.495091,30.730484,3.012723,...,22.395597,27.163926,39.331165,23.304262,7.426222,1,"[36.2185694922, 59.4161536524, 4.3652768555, 0...","[27.2009233742, 9.0176461179, 41.6679538235, 1...",Europe/Berlin,14.5


In [9]:
#convert UTC time to Europe/Berlin
df_all['datetime_loc']=pd.to_datetime(df_all['date']).dt.tz_convert("Europe/Berlin")
df_all[['date','datetime_loc']].head()

,date,datetime_loc
0,2023-05-02T07:00:00.000Z,2023-05-02 09:00:00+02:00
1,2023-05-02T08:00:00.000Z,2023-05-02 10:00:00+02:00
2,2023-05-02T09:00:00.000Z,2023-05-02 11:00:00+02:00
3,2023-05-02T10:00:00.000Z,2023-05-02 12:00:00+02:00
4,2023-05-02T11:00:00.000Z,2023-05-02 13:00:00+02:00


In [11]:
#drop some other cols, keep only the following
df_all_fil=df_all[['segment_id', 'datetime_loc', 'heavy', 'car', 'bike', 'pedestrian']].copy()

In [14]:
# filter to 11 o'clock bucket
df_all_fil_11=df_all_fil[df_all_fil['datetime_loc'].dt.hour==11]

In [15]:
#round and set int for numbers
df_res=df_all_fil_11.round({'heavy': 0, 'car': 0, 'bike': 0, 'pedestrian': 0}).copy()
df_res = df_res.astype({'heavy': 'int', 'car': 'int', 'bike': 'int', 'pedestrian': 'int'})
df_res.head()

,segment_id,datetime_loc,heavy,car,bike,pedestrian
2,9000004672,2023-05-02 11:00:00+02:00,14,21,39,13
2,9000004694,2023-05-02 11:00:00+02:00,18,106,35,15
2,9000004671,2023-05-02 11:00:00+02:00,13,27,60,74


In [16]:
# add names to segment id
df_helper = pd.DataFrame([['Grünauer Straße (Rathaus)', 9000004694], ['Bahnhofstr (Fahrradladen)', 9000004671], ['August-Bebel Allee', 9000004672]], columns=['Name', 'segment_id'])
df_helper

,Name,segment_id
0,Grünauer Straße (Rathaus),9000004694
1,Bahnhofstr (Fahrradladen),9000004671
2,August-Bebel Allee,9000004672


In [17]:
df_countingCompare=df_helper.merge(df_res, on="segment_id")
df_countingCompare

,Name,segment_id,datetime_loc,heavy,car,bike,pedestrian
0,Grünauer Straße (Rathaus),9000004694,2023-05-02 11:00:00+02:00,18,106,35,15
1,Bahnhofstr (Fahrradladen),9000004671,2023-05-02 11:00:00+02:00,13,27,60,74
2,August-Bebel Allee,9000004672,2023-05-02 11:00:00+02:00,14,21,39,13


In [27]:
# save df as csv
df_countingCompare.to_csv("telraam_countingCompare_2023-05-02_eichwalde.csv", encoding="latin1")

In [19]:
# prepare to plot
df_countingCompare_plt=pd.melt(df_countingCompare, id_vars=['Name','segment_id'], value_vars=['heavy', 'car', 'bike', 'pedestrian'], ignore_index=True)
df_countingCompare_plt

,Name,segment_id,variable,value
0,Grünauer Straße (Rathaus),9000004694,heavy,18
1,Bahnhofstr (Fahrradladen),9000004671,heavy,13
2,August-Bebel Allee,9000004672,heavy,14
3,Grünauer Straße (Rathaus),9000004694,car,106
4,Bahnhofstr (Fahrradladen),9000004671,car,27
5,August-Bebel Allee,9000004672,car,21
6,Grünauer Straße (Rathaus),9000004694,bike,35
7,Bahnhofstr (Fahrradladen),9000004671,bike,60
8,August-Bebel Allee,9000004672,bike,39
9,Grünauer Straße (Rathaus),9000004694,pedestrian,15


In [25]:
#plot data witn altair

source = df_countingCompare_plt

alt.Chart(source).mark_bar(
).encode(
    x='variable',
    y=alt.Y('value:Q', stack=None),
    
    color='variable',
    facet='Name',
   
    tooltip=['Name','segment_id','value']
)

alt.Chart(...)